# BULKI_Store Demo

## Starting Server

In [8]:

%env RUST_LOG=debug
print("Let's start")

env: RUST_LOG=debug
Let's start


In [9]:
# import subprocess

# # Start the server as a background process
# server_process = subprocess.Popen(
#     ['/opt/homebrew/bin/mpirun', '-np', '4', 'target/debug/bulkistore-server'],
#     env={'RUST_LOG': 'debug'},
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE
# )

# print("Server started with PID:", server_process.pid)

## Creating Arrays

In [9]:
import bkstore_client as bkc
import numpy as np
import time
from pprint import pprint


bkc.init()
dim_size = 15

print("5D array size = {} elements".format(dim_size**5))

# dim_size = 10
rd1 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)
rd2 = np.random.rand(dim_size, dim_size, dim_size, dim_size, dim_size)

print("rd1.shape=", rd1.shape)
print("rd1 last 2 dim preview:")
print(rd1[0:1, 0:1, 0:1, 0:2, 0:])

print("rd2.shape=", rd2.shape)
print("rd2 last 2 dim preview:")
print(rd2[0:1, 0:1, 0:1, 0:2, 0:])

arr3 = np.random.rand(dim_size, dim_size, dim_size)
print("arr3.shape=", arr3.shape)
print("arr3 last 2 dim preview:")
print(arr3[0:1, 0:2, 0:])

5D array size = 759375 elements
rd1.shape= (15, 15, 15, 15, 15)
rd1 last 2 dim preview:
[[[[[0.91916643 0.57526507 0.02746476 0.36783772 0.33642047 0.10287835
     0.47660514 0.50571474 0.2944028  0.73182231 0.12338597 0.17456935
     0.2120688  0.3522699  0.154965  ]
    [0.60824607 0.49450452 0.34295648 0.36501795 0.16088049 0.44475071
     0.32873376 0.70152553 0.87947118 0.45969996 0.03656584 0.92766225
     0.90900134 0.97332126 0.19243266]]]]]
rd2.shape= (15, 15, 15, 15, 15)
rd2 last 2 dim preview:
[[[[[0.12012631 0.88275265 0.13045821 0.72110563 0.11280704 0.43979397
     0.13140799 0.53699961 0.31499988 0.20651592 0.2443182  0.34088608
     0.80926133 0.71876011 0.63368412]
    [0.39493041 0.66138647 0.13864564 0.05841128 0.18156871 0.87268622
     0.10975273 0.86773428 0.53356156 0.21963713 0.61880254 0.47776777
     0.35828801 0.62576332 0.96915824]]]]]
arr3.shape= (15, 15, 15)
arr3 last 2 dim preview:
[[[0.84684537 0.21955693 0.06038325 0.82745845 0.60366945 0.17512975
   0.

## Client-Side NDArray Arithmetic Operation

In [10]:
arr4 = bkc.polymorphic_add(rd1, rd2)

print("arr4.shape=", arr4.shape)
print("arr4 last 2 dim preview:")
print(arr4[0:1, 0:1, 0:1, 0:2, 0:])

arr4.shape= (15, 15, 15, 15, 15)
arr4 last 2 dim preview:
[[[[[1.03929273 1.45801772 0.15792297 1.08894335 0.44922752 0.54267232
     0.60801314 1.04271435 0.60940268 0.93833823 0.36770417 0.51545543
     1.02133013 1.07103001 0.78864912]
    [1.00317648 1.155891   0.48160212 0.42342923 0.34244919 1.31743693
     0.43848648 1.56925981 1.41303273 0.67933709 0.65536837 1.40543002
     1.26728934 1.59908458 1.1615909 ]]]]]


## Remote Array Slicing

In [11]:
arr5 = bkc.array_slicing(
    arr4, [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 10, -2)]
)

print("arr5.shape=", arr5.shape)
print("arr5 last 2 dim preview:")
print(arr5[0:1, 0:1, 0:1, 0:2, 0:])

arr5.shape= (1, 1, 1, 2, 5)
arr5 last 2 dim preview:
[[[[[0.93833823 1.04271435 0.54267232 1.08894335 1.45801772]
    [0.67933709 1.56925981 1.31743693 0.42342923 1.155891  ]]]]]


## Remote Array Arithmetic Operation

In [12]:
arr6 = bkc.times_two(arr5)

print("arr6.shape=", arr6.shape)
print("arr6:")
print(arr6)

arr6.shape= (1, 1, 1, 2, 5)
arr6:
[[[[[1.87667646 2.0854287  1.08534464 2.17788669 2.91603545]
    [1.35867419 3.13851962 2.63487386 0.84685846 2.31178199]]]]]


## Creating Object with Multiple NDArrays

1. You can attach a series of metadata attributes directly to the object you are creating
2. You can attach a series of NDArrays directly to the object you are creating along with the metadata attributes of these NDArrays.
3. Each of these NDArrays will become a sub-object of the main object you are creating.

In [15]:
# Create an 3D array

name_id = str(np.random.rand(1)[0] * 1000000)[0:6]

gnn_arr1 = np.random.rand(100000, 2)
gnn_arr2 = np.random.rand(1, 100000)

obj_ids = bkc.create_objects(
    obj_name_key="name",
    parent_id=None,
    metadata={
        "name": "container",
        "type": "container",
        "keys": [
            "{}/arr6".format(name_id),
            "{}/arr5".format(name_id),
            "{}/arr4".format(name_id),
            "{}/arr3".format(name_id),
            "{}/gnn_arr1".format(name_id),
            "{}/gnn_arr2".format(name_id),
        ],
        "ranges": [(1, 100), (100, 200)],
        "part_num": 1,
        "part_size": 100,
    },
    data=arr3,
    array_meta_list=[
        {
            "name": "{}/arr6".format(name_id),
            "type": "array",
            "shape": arr6.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr5".format(name_id),
            "type": "array",
            "shape": arr6.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr4".format(name_id),
            "type": "array",
            "shape": arr4.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/arr3".format(name_id),
            "type": "array",
            "shape": arr3.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/gnn_arr1".format(name_id),
            "type": "array",
            "shape": gnn_arr1.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
        {
            "name": "{}/gnn_arr2".format(name_id),
            "type": "array",
            "shape": gnn_arr2.shape,
            "vcount": 100,
            "voffset": 0,
            "vdim": 0,
        },
    ],
    array_data_list=[arr6, arr5, arr4, arr3, gnn_arr1, gnn_arr2],
)
print(obj_ids)

[32119611598135167160399663328550181]


In [15]:
print(f"{name_id}")

276475


## Retrieve Metadata

### Retrieve a single metadata attributes from a single object


In HydraGNN, we need to retrieve a series of metadata attributes from a container. 

If every single metadata retrieval requires a single request, that would be too much overhead.

MetaPS is a new metric we designed to calculate the number of metadata attributes we can retrieve from one object within a second.


In [16]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size", "title"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_ids[0],
        meta_keys=[meta_keys[k]],  # retreiving SINGLE!!!
    )
    if i < len(meta_keys):
        print(result)

print(
    "Single Metadata retrieval of single object : TPS = {} MetaPS: {}".format(
        1000 / (time.time() - start), 1000 / (time.time() - start)
    )
)

{'obj_id': 32119611598135167160399663328550181, 'metadata': {'type': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'keys': ['8839.0/arr6', '8839.0/arr5', '8839.0/arr4', '8839.0/arr3', '8839.0/gnn_arr1', '8839.0/gnn_arr2']}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'ranges': [(1, 100), (100, 200)]}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'name': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'part_num': 1}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'part_size': 100}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {}, 'sub_obj_metadata': None}
Single Metadata retrieval of single object : TPS = 1207.5923666346316 MetaPS: 1207.5909759126957


In [17]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size", "title"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_id="container",
        meta_keys=[meta_keys[k]],  # retreiving SINGLE!!!
    )
    if i < len(meta_keys):
        print(result)

print(
    "Single Metadata retrieval of single object : TPS = {} MetaPS: {}".format(
        1000 / (time.time() - start), 1000 / (time.time() - start)
    )
)

{'obj_id': 32119611598135167160399663328550181, 'metadata': {'type': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'keys': ['8839.0/arr6', '8839.0/arr5', '8839.0/arr4', '8839.0/arr3', '8839.0/gnn_arr1', '8839.0/gnn_arr2']}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'ranges': [(1, 100), (100, 200)]}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'name': 'container'}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'part_num': 1}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {'part_size': 100}, 'sub_obj_metadata': None}
{'obj_id': 32119611598135167160399663328550181, 'metadata': {}, 'sub_obj_metadata': None}
Single Metadata retrieval of single object : TPS = 1342.2137882150394 MetaPS: 1342.2120701368578


### Retrieving multiple metadata attributes from a single object

Now we can combine the metadata retrieval of a single container into one single request. 

MetaPS is a new metric we designed to calculate the number of metadata attributes we can retrieve from one object within a second.

Slight drop on TPS but MEGA IMPROVEMENT for MetaPS

In [21]:
meta_keys = ["type", "keys", "ranges", "name", "part_num", "part_size"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        meta_keys=meta_keys,  # retrieving MULTIPLE!!!
    )
pprint(result)
print(
    "\n Multiple Metadata retrieval of single object : TPS={}, MetaPS: {} ".format(
        1000 / (time.time() - start), 1000 * len(meta_keys) / (time.time() - start)
    )
)

{'metadata': {'keys': ['8839.0/arr6',
                       '8839.0/arr5',
                       '8839.0/arr4',
                       '8839.0/arr3',
                       '8839.0/gnn_arr1',
                       '8839.0/gnn_arr2'],
              'name': 'container',
              'part_num': 1,
              'part_size': 100,
              'ranges': [(1, 100), (100, 200)],
              'type': 'container'},
 'obj_id': 32119611598135167160399663328550181,
 'sub_obj_metadata': None}

 Multiple Metadata retrieval of single object : TPS=1203.8663294324845, MetaPS: 7223.179317556042 


### Retrieving single attributes from a single sub-object

In [22]:
meta_keys = ["name", "type", "shape", "vcount", "voffset", "vdim"]

start = time.time()
for i in range(1000):
    k = i % len(meta_keys)
    result = bkc.get_object_metadata(
        obj_id="8839.0",
        meta_keys=[meta_keys[k]],  # retreiving SINGLE!!!
    )
    if i < len(meta_keys):
        print(result)

print(
    "Single Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 / (time.time() - start)
    )
)

ValueError: Failed to get object metadata: Connection Failure: h2 protocol error: http2 error

### Retrieving multiple attributes from single sub-objects

In [20]:
meta_keys = ["name", "type", "shape", "vcount", "voffset", "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[1],
        meta_keys=meta_keys,  # retreiving SINGLE!!!
    )

pprint(result)
print(
    "\n Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 * len(meta_keys) / (time.time() - start)
    )
)

{'metadata': {'name': '276475/arr6',
              'shape': [1, 1, 1, 2, 5],
              'type': 'array',
              'vcount': 100,
              'vdim': 0,
              'voffset': 0},
 'obj_id': 32119348377435593173945324017138841,
 'sub_obj_metadata': None}

 Multiple Metadata retrieval of a single sub-object : TPS=1227.8993406301297, MetaPS: 7367.376632318905


### Retrieving multiple attributes from multiple sub-objects

In [21]:
meta_keys = ["name", "type", "shape", "vcount", "voffset", "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        sub_meta_keys=meta_keys,  # retreiving MULTIPLE for every sub-object!!!
    )

pprint(result)
print(
    "Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 * len(meta_keys) * 3 / (time.time() - start)
    )
)

{'metadata': {},
 'obj_id': 32119348377435464046736803755310233,
 'sub_obj_metadata': [{'metadata': {'name': '276475/arr3',
                                    'shape': [15, 15, 15],
                                    'type': 'array',
                                    'vcount': 100,
                                    'vdim': 0,
                                    'voffset': 0},
                       'name': '276475/arr3',
                       'obj_id': 32119348377447159282479552790903961},
                      {'metadata': {'name': '276475/arr6',
                                    'shape': [1, 1, 1, 2, 5],
                                    'type': 'array',
                                    'vcount': 100,
                                    'vdim': 0,
                                    'voffset': 0},
                       'name': '276475/arr6',
                       'obj_id': 32119348377435593173945324017138841},
                      {'metadata': {'name': '276475/arr5',

### Retrieving different set of attributes from multiple sub-objects

In [22]:
# meta_keys=["name",
#             "type",
#             "shape",
#             "vcount",
#             "voffset",
#             "vdim"]

start = time.time()
for i in range(1000):
    result = bkc.get_object_metadata(
        obj_ids[0],
        sub_meta_keys={
            "{}/arr5".format(name_id): ["name", "type", "voffset"],
            "{}/arr4".format(name_id): ["type", "vcount"],
        },  # retrieving different set of metadata for different sub-objects
    )

pprint(result)
print(
    "\n Multiple Metadata retrieval of a single sub-object : TPS={}, MetaPS: {}".format(
        1000 / (time.time() - start), 1000 * 5 / (time.time() - start)
    )
)

{'metadata': {},
 'obj_id': 32119348377435464046736803755310233,
 'sub_obj_metadata': [{'metadata': {'name': '276475/arr5',
                                    'type': 'array',
                                    'voffset': 0},
                       'name': '276475/arr5',
                       'obj_id': 32119348377435832981618286536277145},
                      {'metadata': {'type': 'array', 'vcount': 100},
                       'name': '276475/arr4',
                       'obj_id': 32119348377435999002314954217208985}]}

 Multiple Metadata retrieval of a single sub-object : TPS=1184.3237843207348, MetaPS: 5921.603873131561


### Retrieving array of a single object

In [23]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[slice(0, 1), slice(0, 1), slice(0, 2, -1)],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s = {}".format(
        1000 / (time.time() - start), (1000 / 6) / (time.time() - start)
    )
)

{'array_slice': array([[[0.48205712, 0.67145357]]]), 'sub_obj_slices': []}

 retrieving array slice of a single 3D array, TPS=1000.2439617092201, Sample/s = 166.70716792840616


### Retrieving array slices of multiple sub-objects (Large subarray included)

In [24]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=[slice(0, 1), slice(0, 2), slice(0, 2, -1)],
        sub_obj_regions=[
            (
                "{}/arr5".format(name_id),
                [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 2, -1)],
            ),
            (
                "{}/arr4".format(name_id),
                [
                    slice(0, 2, -1),
                    slice(0, 1),
                    slice(0, 1),
                    slice(0, 1),
                    slice(0, 2, -1),
                ],
            ),
            (
                "{}/arr3".format(name_id),
                [slice(0, 1), slice(0, 2), slice(0, 10, -2)],
            ),
            (
                "{}/arr6".format(name_id),
                [slice(0, 1), slice(0, 1), slice(0, 1), slice(0, 2), slice(0, 2, -1)],
            ),
        ],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s={}".format(
        1000 / (time.time() - start), (1000 / (4 / 6)) / (time.time() - start)
    )
)

{'array_slice': array([[[0.48205712, 0.67145357],
        [0.83928621, 0.8233581 ]]]),
 'sub_obj_slices': [{'array': array([[[[[1.29210399, 0.83085831],
          [0.59747627, 1.57740461]]]]]),
                     'id': 32119348377435832981618286536277145,
                     'name': '276475/arr5'},
                    {'array': array([[[[[0.73377526, 1.54474945]]]],



       [[[[1.30933282, 0.33224046]]]]]),
                     'id': 32119348377435999002314954217208985,
                     'name': '276475/arr4'},
                    {'array': array([[[0.70220418, 0.42578952, 0.5314196 , 0.41273197, 0.48205712],
        [0.46147236, 0.18501475, 0.72301612, 0.88348578, 0.83928621]]]),
                     'id': 32119348377447159282479552790903961,
                     'name': '276475/arr3'},
                    {'array': array([[[[[2.58420797, 1.66171663],
          [1.19495253, 3.15480921]]]]]),
                     'id': 32119348377435593173945324017138841,
                     '

### Retrieving slices of multiple objects (only GNN arrays)

In [25]:
start = time.time()
for i in range(1000):
    result = bkc.get_object_data(
        obj_ids[0],
        region=None,
        sub_obj_regions=[
            (
                "{}/gnn_arr1".format(name_id),
                [slice(0, 20), slice(0, 1)],
            ),
            (
                "{}/gnn_arr1".format(name_id),
                [
                    slice(0, 10, -1),
                    slice(0, 1),
                ],
            ),
            (
                "{}/gnn_arr1".format(name_id),
                [
                    slice(0, 15, -1),
                    slice(0, 2),
                ],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [
                    slice(0, 1),
                    slice(0, 20, -1),
                ],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [
                    slice(0, 1),
                    slice(0, 20, -1),
                ],
            ),
            (
                "{}/gnn_arr2".format(name_id),
                [
                    slice(0, 1),
                    slice(0, 20, -1),
                ],
            ),
        ],
    )
pprint(result)
print(
    "\n retrieving array slice of a single 3D array, TPS={}, Sample/s={}".format(
        1000 / (time.time() - start), (1000) / (time.time() - start)
    )
)

{'array_slice': None,
 'sub_obj_slices': [{'array': array([[0.4725509 ],
       [0.4009871 ],
       [0.28816418],
       [0.03304448],
       [0.96981313],
       [0.91304028],
       [0.2036412 ],
       [0.24010823],
       [0.60022992],
       [0.71936996],
       [0.98409307],
       [0.82907341],
       [0.75973639],
       [0.9982704 ],
       [0.49448435],
       [0.67091999],
       [0.31854262],
       [0.21357598],
       [0.71976736],
       [0.88784411]]),
                     'id': 32119348377447399090152515310042265,
                     'name': '276475/gnn_arr1'},
                    {'array': array([[0.71936996],
       [0.60022992],
       [0.24010823],
       [0.2036412 ],
       [0.91304028],
       [0.96981313],
       [0.03304448],
       [0.28816418],
       [0.4009871 ],
       [0.4725509 ]]),
                     'id': 32119348377447399090152515310042265,
                     'name': '276475/gnn_arr1'},
                    {'array': array([[0.49448435, 0.498407

## Closing the Store

In [26]:
# server_process.terminate()
# server_process.wait()
# print("Server process terminated.")